# Ejercicio 1 - PCA

Tomar el sample dataset de `diabetes` de la librería `sklearn` y realizar la reducción de dimensionalidad correspondiente para obtener la extracción de características `feature extraction`. Describir el proceso y su resultado

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn.decomposition import PCA
from sklearn.datasets import load_diabetes
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [ ]:
X, y = #TODO
print(X.shape)
print(y.shape)

In [ ]:
pca=PCA() #TODO
pca.fit() #TODO
a,b = #TODO pca.explained_variance_
a = a.round(5) # Rendoar valores para poder mostrarlos como título
b = b.round(5) # Rendoar valores para poder mostrarlos como título
Z=pca.transform(X)
plt.title('Explained Variance:' + str(a) + ',' + str(b))
plt.axis('equal')
plt.scatter(Z[:,0], Z[:,1]);

In [ ]:
rng=np.random.RandomState(0)
X=rng.randn(3,400)
p=rng.rand(10,3)  # Random projection into 10d
X=np.dot(p, X)
print(X)

In [ ]:
pca=PCA()
pca.fit() #TODO
v=#TODO pca.explained_variance_
print(v)
plt.plot(np.arange(1,11), np.cumsum(v));

# Ejercicio 2 Segmentación de imágenes con K-Means

Hemos hablado de los beneficios de varios algoritmos para segmentación de imágenes, como es el caso de `meanshift`.
Para el siguiente ejemplo, debes implementar segmentación de imágenes utilizando nuestro algoritmo más básico de agrupamiento.
Utilizar la imágen `./data/montevideo.jpg`

In [ ]:
from skimage.color import rgb2gray
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
%matplotlib inline
from scipy import ndimage
import warnings
warnings.filterwarnings("ignore")

In [ ]:
pic = plt.imread()/255  #TODO dividing by 255 to bring the pixel values between 0 and 1
print(pic.shape)
plt.imshow() #TODO

In [ ]:
pic_n = pic.reshape() #TODO
pic_n.shape

In [ ]:
#TODO - Plot del codo

In [ ]:
# Tomar el valor k resultante del plot anterior.
# Probar otros valores de K para ver su resultado
kmeans = KMeans().fit() #TODO
pic2show = kmeans.cluster_centers_[] #TODO

In [ ]:
cluster_pic = pic2show.reshape() #TODO
plt.imshow() #TODO

# Ejercicio 3 - GAN

Importar el dataset de `CIFAR10 data` y construir un generador.

[Información del dataset](https://www.cs.toronto.edu/~kriz/cifar.html)

>Note: Se recomienda ejecutar todo el notebook o a partir de este ejercicio en COLAB

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from os import listdir, makedirs
from os.path import join, exists, expanduser
import keras
from keras import layers
import os
#from keras.preprocessing import image
from keras.utils import load_img, img_to_array
import warnings
warnings.filterwarnings("ignore")

In [ ]:
latent_dim = 32
height = 32
width = 32
channels = 3

generator_input = keras.Input(shape=(latent_dim,))

# First, transform the input into a 16x16 128-channels feature map
x = layers.Dense(128 * 16 * 16)(generator_input)
x = layers.()()#TODO Completar la función de activación luego del . y el último paréntisis 
x = layers.Reshape((16, 16, 128))()#TODO

# Then, add a convolution layer
x = layers.Conv2D(256, 5, padding='same')()#TODO
x = layers.()() #TODO Completar la función de activación luego del . y el último paréntisis

# Upsample to 32x32
x = layers.Conv2DTranspose(256, 4, strides=2, padding='same')()#TODO
x = layers.()()#TODO Completar la función de activación luego del . y el último paréntisis

# Few more conv layers
x = layers.Conv2D(256, 5, padding='same')()#TODO
x = layers.()() #TODO Completar la función de activación luego del . y el último paréntisis
x = layers.Conv2D(256, 5, padding='same')()#TODO
x = layers.()()#TODO Completar la función de activación luego del . y el último paréntisis

# Produce a 32x32 1-channel feature map
x = layers.Conv2D(channels, 7, activation='tanh', padding='same')()#TODO
generator = keras.models.Model(generator_input, x)
generator.summary()

In [ ]:
discriminator_input = layers.Input(shape=(height, width, channels))
x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.()()#TODO Completar la función de activación luego del . y el último paréntisis
x = layers.Conv2D(128, 4, strides=2)()#TODO
x = layers.()()#TODO Completar la función de activación luego del . y el último paréntisis
x = layers.Conv2D(128, 4, strides=2)()#TODO
x = layers.()()#TODO Completar la función de activación luego del . y el último paréntisis
x = layers.Conv2D(128, 4, strides=2)()#TODO
x = layers.()()#TODO Completar la función de activación luego del . y el último paréntisis
x = layers.Flatten()() #TODO

# One dropout layer - important trick!
x = layers.Dropout(0.4)()#TODO

# Classification layer
x = layers.Dense(1, activation='')() #TODO

discriminator = keras.models.Model(discriminator_input, x)
discriminator.summary()

# To stabilize training, we use learning rate decay
# and gradient clipping (by value) in the optimizer.
discriminator_optimizer = keras.optimizers.RMSprop(lr=0.0008, clipvalue=1.0, decay=1e-8)
discriminator.compile(optimizer=discriminator_optimizer, loss='binary_crossentropy')

In [ ]:
# Set discriminator weights to non-trainable
# (will only apply to the `gan` model)
discriminator.trainable = False

gan_input = keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input, gan_output)

gan_optimizer = keras.optimizers.RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')
gan.summary()

In [ ]:
# Load CIFAR10 data
(x_train, y_train), (_, _) = #TODO

# Select frog images (class 6)
x_train = x_train[y_train.flatten() == 6]

# Normalize data
x_train = x_train.reshape(
    (x_train.shape[0],) + (height, width, channels)).astype('float32') / 255.

iterations = 1000
batch_size = 20
save_dir = '.data/output/gan_images/'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Start training loop
start = 0
for step in range(iterations):
    # Sample random points in the latent space
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))

    # Decode them to fake images
    generated_images = generator.predict(random_latent_vectors)

    # Combine them with real images
    stop = start + batch_size
    real_images = x_train[start: stop]
    combined_images = np.concatenate([generated_images, real_images])

    # Assemble labels discriminating real from fake images
    labels = np.concatenate([np.ones((batch_size, 1)),
                             np.zeros((batch_size, 1))])
    # Add random noise to the labels - important trick!
    labels += 0.05 * np.random.random(labels.shape)

    # Train the discriminator
    d_loss = discriminator.train_on_batch(combined_images, labels)

    # sample random points in the latent space
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))

    # Assemble labels that say "all real images"
    misleading_targets = np.zeros((batch_size, 1))

    # Train the generator (via the gan model,
    # where the discriminator weights are frozen)
    a_loss = gan.train_on_batch(random_latent_vectors, misleading_targets)
    
    start += batch_size
    if start > len(x_train) - batch_size:
      start = 0

    # Occasionally save / plot
    if step % 100 == 0:
        # Save model weights
        gan.save_weights('.data/output/gan.h5')

        # Print metrics
        print('discriminator loss at step %s: %s' % (step, d_loss))
        print('adversarial loss at step %s: %s' % (step, a_loss))

In [ ]:
import matplotlib.pyplot as plt

# Sample random points in the latent space
random_latent_vectors = np.random.normal(size=(10, latent_dim))

# Decode them to fake images
generated_images = generator.predict(random_latent_vectors)

for i in range(generated_images.shape[0]):
    img = image.array_to_img(generated_images[i] * 255., scale=False)
    plt.figure()
    plt.imshow(img)
    
plt.show()


# Ejercicio 4 - VAE (Opcional)

Tomando cualquier dataset que desees, implementa tu `VAE`

In [ ]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')
import keras
from keras import layers
from keras import backend as K
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt